In [ ]:
! python3 --version

In [ ]:
import gensim
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import os
import string

In [ ]:
N = 10000
ETD_PATH = '/mnt/ceph/cme/thesis/'

list_of_docs = os.listdir(ETD_PATH)[:N]
len(list_of_docs)

In [ ]:
def data_generator():
    
    for idx, folder in enumerate(list_of_docs):
        for _ in os.listdir(ETD_PATH + folder):
            file_name = ''
            if _.endswith('.pdf.txt') or _.endswith('.PDF.txt'):
                file_name = _
                break
        if file_name:
            with open(ETD_PATH + folder + '/' + file_name) as f:
                document = f.read()
                
                tokens = nltk.word_tokenize(document.lower().translate(str.maketrans('', '', string.punctuation)))
                stems = []
                
                for item in tokens:
                    if item in stopwords.words():
                        continue
                    stems.append(PorterStemmer().stem(item))

                #tokens = gensim.utils.simple_preprocess(document)
                
                yield gensim.models.doc2vec.TaggedDocument(stems, [int(folder) + 1])

In [ ]:
train_corpus = list(data_generator())

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=128, min_count=2, epochs=20)
model.build_vocab(train_corpus)

In [ ]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.save('../obj/doc2vec1')